In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

class spotWrapper():
    def __init__(self):
        client_id = 'accd0aa479164ddcbf1cbf822512b80b'
        client_secret = '58bfc467435045e7b61c86fb03385729'
        client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
        self.spot = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    def getAlbumsTracks(self, id):
        return self.spot.album_tracks(id)
    def getAudioFeatures(self, song_id):
        return self.spot.audio_features(song_id)
def readArtists():
    artists = []
    with open('/Users/garcgabe/Desktop/HipHop-Analysis-Project/artists.txt') as file:
        name = 'x'
        while(name != ''):
            name = file.readline()
            if name.strip() and 'stop' not in name:
                artists.append(name.split("\n")[0])  
            else:
                break
    return artists

spot = spotWrapper()
artists = readArtists()

In [9]:
alb = spot.getAlbumsTracks("spotify:album:3MATDdrpHmQCmuOcozZjDa")["items"]#[2].keys()
for i in range(0,15):
    title = alb[i]["name"]
    #print(alb[i].keys())
    print(alb[i]["name"])
    print(alb[i]["explicit"])
alb = spot.getAlbumsTracks("spotify:album:4McL6uB0omKrLAsZtXQwDA")["items"]#[2].keys()
for i in range(0,15):
    title = alb[i]["name"]
    #print(alb[i].keys())
    print(alb[i]["name"])
    print(alb[i]["explicit"])


Distorted Records
True
A$AP Forever REMIX (feat. Moby, T.I. & Kid Cudi)
True
Tony Tone
True
Fukk Sleep (feat. FKA twigs)
True
Praise The Lord (Da Shine) (feat. Skepta)
True
CALLDROPS (feat. Kodak Black)
True
Buck Shots
True
Gunz N Butter (feat. Juicy J)
True
Brotha Man (feat. French Montana)
True
OG Beeper
True
Kids Turned Out Fine
True
Hun43rd
True
Changes
True
Black Tux, White Collar
True
Purity (feat. Frank Ocean)
True
Distorted Records
False
A$AP Forever REMIX (feat. Moby, T.I. & Kid Cudi)
False
Tony Tone
False
F**k Sleep (feat. FKA twigs)
False
Praise The Lord (Da Shine) (feat. Skepta)
False
CALLDROPS (feat. Kodak Black)
False
Buck Shots
False
Gunz N Butter (feat. Juicy J)
False
Brotha Man (feat. French Montana)
False
OG Beeper
False
Kids Turned Out Fine
False
Hun43rd
False
Changes
False
Black Tux, White Collar
False
Purity (feat. Frank Ocean)
False
